In [2]:
import pandas as pd
import numpy as np

In [90]:
import re
import string

punctuation = string.punctuation

def process_string(text):
    text = text.translate(str.maketrans("", "", punctuation))
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.lower()

In [91]:
import requests
from bs4 import BeautifulSoup

class WebScraper:
    def __init__(self, url) -> None:
        self.url = url
        self.soup = self.get_html_content()

    def get_html_content(self):
        try:
            # Make the HTTP request
            response = requests.get(self.url)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the HTML content
                soup = BeautifulSoup(response.content, 'html.parser')
                return soup
            else:
                return f"Failed to retrieve the page. Status code: {response.status_code}"
        except requests.exceptions.RequestException as e:
            return f"An error occurred: {e}"
    
    def get_text_by_classname(self, class_name="message-body js-selectToQuote"):
        texts = []
        try:
            elements = self.soup.find_all(class_=class_name)
            for element in elements:
                texts.append(process_string(element.get_text().replace('\n', ' ').replace('\t', ' ').replace('\xa0', '')))

            return texts

        except Exception as e:
            return f"An error occurred while fetching the elements: {e}"
    

In [92]:
scrapper = WebScraper("https://forum.adrenaline.com.br/threads/indicacoes-de-aquisicao-de-pc-novo-configuracao-completa-gamer-profissional-uso-casual.691749/")

In [93]:
scrapper.get_text_by_classname()

['tópico para concentrar mensagens de quem busca dicas e indicações para montar um pc novo',
 'pessoal uma dúvida sobre config to pensando em montar um pc com as especificações abaixo comprando peça por peça para dar uma economizada i5 11400f 16gb ram 2x8gb 3200 placa mãe h510mt m2 v20 rtx 3050 8gb fonte corsair 500w 80 plus white busco gastar no máximo 67k comprando uma peça de cada vez acompanhando as promoções etc vocês mudariam algo pc para uso no dia a dia trabalho homeoffice e jogos em full hd vi algumas pessoas falando mal da 11ª geração da intel pensei em talvez pegar um i3 12100f mas realmente não sei qual seria melhor edit pro meu trabalho não preciso de tanta coisa uso apenas o google chrome com várias abas abertas rsrs',
 'senhores deem uma luz aqui por favor estou querendo montar um pc para jogar em 4k e 60fps mas dependendo do orçamento final eu fico com o quadhd a ideia é montar um em torno do amd ryzen 5600x e da rtx3080 o que eu posso comprar de placa mãe cooler vram e

In [48]:

links = ['https://forum.adrenaline.com.br/threads/indicacoes-de-aquisicao-de-pc-novo-configuracao-completa-gamer-profissional-uso-casual.691749/']

In [96]:


comments = []
for i in range(1, 50):
    comments.extend(WebScraper(links[0] + f'page-{i}').get_text_by_classname())


In [98]:
df = pd.DataFrame(comments, columns=['comments'])

In [109]:
from textblob import TextBlob
import nltk

In [113]:
# nltk.download()

In [126]:
text = df['comments'][5]

blob = TextBlob(text)
blob.tags  # [('The', 'DT'), ('titular', 'JJ'),
#  ('threat', 'NN'), ('of', 'IN'), ...]

blob.noun_phrases  # WordList(['titular threat', 'blob',
#            'ultimate movie monster',
#            'amoeba-like mass', ...])

for sentence in blob.sentences:
    print(sentence.sentiment)
# 0.060
# -0.341

Sentiment(polarity=0.35, subjectivity=0.55)


In [127]:
df['comments'][5]

'boa tarde pessoal tenho uma máquina frankenstein aqui i5 2310 12 gb ram genéricas rx 480 8 gb da msi fonte evga 650 w quero dar um up inicialmente na cpu queria opções para fazer essa mudança a placa de vídeo ta velhinha mas diante dos preços atuais estou me agarrando a ela mesma tenho preferência por processadores com vídeo integrado pois já tive a experiência de ter uma placa de vídeo queimada e ficar com o pc inutilizado por um tempo a finalidade do pc será para jogos mesmo porém me contento com um bom full hd com isso quais cpu mobo memoria posso estar pegando para ter uma máquina rodando os títulos atuais em 60 fps ps tenho disponível 3500 para atualizar'

In [131]:
df['sentiment'] = df['comments'].apply(lambda x: TextBlob(x))

In [139]:
df.sentiment[3].sentiment_assessments

Sentiment(polarity=0.0, subjectivity=0.0, assessments=[])

In [130]:
df.to_csv('comments.csv', index=False)

In [140]:
def to_dict(df, i):
    row = df.loc[i]
    return {
        'comment': row['comments'],
        'sentiment': row['sentiment'].sentiment,
        'sentiment_assessments': row['sentiment'].sentiment_assessments
    }

In [141]:
to_dict(df, 5)

{'comment': 'boa tarde pessoal tenho uma máquina frankenstein aqui i5 2310 12 gb ram genéricas rx 480 8 gb da msi fonte evga 650 w quero dar um up inicialmente na cpu queria opções para fazer essa mudança a placa de vídeo ta velhinha mas diante dos preços atuais estou me agarrando a ela mesma tenho preferência por processadores com vídeo integrado pois já tive a experiência de ter uma placa de vídeo queimada e ficar com o pc inutilizado por um tempo a finalidade do pc será para jogos mesmo porém me contento com um bom full hd com isso quais cpu mobo memoria posso estar pegando para ter uma máquina rodando os títulos atuais em 60 fps ps tenho disponível 3500 para atualizar',
 'sentiment': Sentiment(polarity=0.35, subjectivity=0.55),
 'sentiment_assessments': Sentiment(polarity=0.35, subjectivity=0.55, assessments=[(['full'], 0.35, 0.55, None)])}

In [151]:
from deep_translator import GoogleTranslator


In [156]:
df.comments[0]

'tópico para concentrar mensagens de quem busca dicas e indicações para montar um pc novo'

In [157]:
translated = GoogleTranslator(source='auto', target='en').translate(df.comments[0])  # output -> Weiter so, du bist großartig


In [158]:
translated

'topic to concentrate messages from those looking for tips and instructions for building a new PC'

In [178]:
def divide_string_into_chunks(string, chunk_size):
    # Using list comprehension to create chunks
    return [string[i:i + chunk_size] for i in range(0, len(string), chunk_size)]

def translate_comment(comment):
    if len(comment) >= 4999:
        translation = ""
        for chunk in divide_string_into_chunks(comment, 4999):
            translation += GoogleTranslator(source='auto', target='en').translate(chunk)
    else:
        return GoogleTranslator(source='auto', target='en').translate(comment)

In [179]:
translate_comment("viniciusbizinotto disse boa tarde galera kit com 2500 qual comprar no caso o processo ja vai ser o amd ryzen 5 5600g mas minha maior duvida é qual placa mae pegar uma que seja com a bios ja atualizada clique para expandir pega duas memórias de 3200mhz ballistix de 8gb totalizando 16gb ryzen 5 5600g ok sobre placamãe placamãe msi mag b550m bazooka amd am4 matx encontre as melhores ofertas aqui no kabum confira nossa linha de computadores eletrônicos celulares notebooks tablets câmeras digitais e mais compre já wwwkabumcombr httpswwwterabyteshopcombrproduto13983placamaeasustufgamingb550mpluschipsetb550amdam4matxddr4 ou tentaria contato com a loja e verificaria se é possível atualização de bios para essa aqui httpswwwkabumcombrproduto100672placamaeasrockb450msteellegendamdam4matxddr490mxb9y0a0bayz enfim boa sorte ysbrigolino disse fala pessoal tudo bem como o titulo deste topico diz estou ha 4 anos desconectado do mundo dos games em 2018 vendi meu setup que listarei na sequencia duvida estou pretendendo remontar um setup para jogar owow2 com meus filhos e poder usufluir de uns 144hzfps voces acreditam que a melhor estrategia seria repetir o meu antigo setup ou ja existem hardware mais modernos eou com melhor custo beneficio para substituir obs terei q comprar tudo inclusive perifericos e monitor acredito que neles existam opcoes com melhor custo beneficio atualmente setup antigo ryzen 2600x corsair vengeance lpx ddr4 16gb amd am4 b450 aorus evga gtx 1080 ssd crucial 128gb os western digital black caviar 2tb corsair cx 500 benq xl2420z ducky one coolermaster haf 932 asus xonar dgx sennheiser hd 598 logitech g203 pad steelserires qck black muito obrigado pelo suporte de sempre clique para expandir sim de fato existem opções com melhor custo benefício atualmente poderia informar qual seu orçamento nandomc disse pessoal tô com 3k pra montar um pc para trabalharprogramação e estudar não preciso de vga qual seria a melhor configuração nessa faixa de preço clique para expandir cpu ryzen 5600g ou 5700g dependendo do quanto você precisará de mais núcleos e é claro da diferença do preço memórias ballistix 3200mhz 2x8gb ou 2x16gb dependendo de quais programas você utilizará para trabalhar se realmente 16gb da conta ou se precisa de 32gb motherboard b450m steel legend ou b450m aorus fonte corsair cv450w armazenamento ssd pro sistema operacional e hd para arquivar documentos e afins gabinete sua escolha enzo assimos disse eu não sei como postar aqui então vou falar por aqui mesmo eu tô querendo comprar um pc de 4k com foco em jogos full hd assima de 70 fps o que vocês recomendam clique para expandir pegaria esse kit httpswwwgkinfostorecombrkitmaeasusprimeb660mkd4ddr4lga1700matx 2x8gb ddr4 8gb 3200mhz ballistix ssd m2 320gb ou similar para o windows programas e hd 1tb ou 2tb para jogos fonte corsair cv450w gabinete sua escolha gpu pra essa faixa de preço o que tem é 6500xt ou gtx 1650 como essa mobo tem suporte para pcie 40 a 6500xt se sairá melhor com relação à gtx 1650 porém você pode tentar economizar e tentar algo como 1660ti ou rtx3050 mas creio que já ficará acima dos 5k fehver disse notebook tá morrendo preciso de um pc novo resolvi montar um desktop ele será usado para jogos em geral edição de video e programação com meu baixo conhecimento em hardwares fiz várias pesquisas e essa é o resultado delas placamãe gigabyte aorus b450m cpu ryzen 5 5500 cooler cooler box memória ram 2x8 crucial ballistix 3200mhz fonte corsair cv550 80 plus bronze discos 1x ssd kingston nv1 250gb m2 2280 nvme 1x ssd kingston a400 480gb sata vga asus geforce gtx 1650 dual 4gb placa wireless wifi fenvi gostaria de saber a opinião de vocês se é um bom pc para os usos citados e se a fonte vai dar conta de tudo clique para expandir parece bacana pra mim apesar de achar o preço das gtx 1650 surreais assim como as outras gpus talvez tentaria fazer um esforço pra tentar uma rtx 3050 pelo menos e sobre a fonte ela vai dar conta de sobra capivas disse notebook pra arquiteto pra usar somente o lumion 11 tava pensando no nitro 5 com a 1650 que ta em uns 45k acham que vale a pena o budget é 6k estourado então nao posso considerar alienware g15 predator etc fazer o esforço pra pegar algo com uma 3050 seria valioso ou nao clique para expandir esse nitro 5 com gtx 1650 parece o mínimo para o lumion 11 tenso que fazer esforço para pegar algo com 3050ti já sobe bastante o orçamento acima dos 78k se é algo urgente para ti acho que possa valer a pena o nitro 5 porém se não for talvez seja válido esperar algo com em promoção com pelo menos uma 1660ti daí seria legal uns updates de memória creio que precise para o lumion 11 16gb seria o mínimo se não me engano e ssd já fica coisa linda jdutra disse bom dia a todos estou montando um novo pc gamer para jogar em 2k ainda estou analisando qual monitor comprar o que me dizem da seguinte configuracao mudariam algo amd ryzen 7 5800x3dplacamãe asus rog strix x570f gaming32gb ram ddr4 cl16water cooler nzxt kraken x73 360mm rgb rlkrx7301fonte tentar com atualgalax gtx 3800 12gbgabinete gamer lian li o11 dynamic evo cinza o11deg obs quanto a fonte tentarei utilizar minha atual corsair 650w caso necessario comprarei outra mouse e teclado ja tenho penso em gastar no maximo r 20k com tudo inclusive monitor att jonas clique para expandir rtx 3080 geralmente requer pelo menos uma fonte de 750w e como você está pegando uma placamãe para overclock recomendaria pegar uma 850w tipo essa httpswwwkabumcombrproduto103282fontexpgcorereactor850w80plusgoldmodular e sobre as memórias tentaria pelo menos 3600mhz ou acima e 32gb parece bem legal pra mim")

In [180]:
df['comments_en'] = df['comments'].apply(translate_comment)